# Part 1: Conceptes Fonamentals d'Agents d'IA amb CrewAI
## JCM Technologies - Departament de R+D

<table style="margin: 0; text-align: left; width:100%; border: 2px solid #2e86ab;">
    <tr>
        <td style="width: 150px; vertical-align: middle; padding: 20px;">
            <img src="https://raw.githubusercontent.com/googlefonts/noto-emoji/main/png/128/emoji_u1f9e0.png" width="120" style="display: block;" />
        </td>
        <td style="padding: 20px;">
            <h2 style="color:#2e86ab; margin: 0;">Objectius d'aprenentatge</h2>
            <ul style="color:#333; margin-top: 10px;">
                <li>Entendre la diferència entre <strong>workflows</strong> i <strong>agents</strong></li>
                <li>Conèixer les diferents <strong>capes de frameworks</strong> per agents d'IA</li>
                <li>Crear el teu primer <strong>Crew</strong> amb CrewAI i Google Gemini</li>
                <li>Aplicar-ho a un cas real: analitzar un BOM (Bill of Materials)</li>
            </ul>
            <p style="color:#ff7800; margin: 10px 0 0 0;"><strong>Durada estimada:</strong> 50-60 minuts</p>
        </td>
    </tr>
</table>

---
## 📚 Continguts

1. **Recordatori ràpid:** Què són els LLMs? (5 min)
2. **Costos i models:** Google Gemini (5 min)
3. **Frameworks d'agents:** De baix a alt nivell (5 min)
4. **Workflows vs Agents:** La diferència clau (15 min)
5. **Mans a l'obra:** El teu primer Crew (15 min)
6. **Cas pràctic:** Crew analitzador de BOM (15 min)

---
## 1. Recordatori Ràpid: Què són els LLMs?

Ja coneixes ChatGPT i Copilot, així que aquest serà ràpid. Però anem a repassar els conceptes bàsics perquè són la base dels agents.

### 1.1 Els components bàsics

Un **Large Language Model (LLM)** és, essencialment:

```
Input (prompt) → [Model] → Output (resposta)
```

**Conceptes que ja coneixes:**
- **Prompt:** Les instruccions que dones al model
- **Tokens:** Les unitats que el model processa (aproximadament 4 caràcters per token)
- **Temperature:** Controla la "creativitat" (0 = determinista, 1 = creatiu)
- **Context window:** Quanta informació pot "recordar" el model

**El que potser no saps:** Quan cridem un LLM programàticament, no estem parlant amb ChatGPT o Gemini directament. Estem cridant una **API** que ens dóna accés al model.

### 1.2 Exemple ràpid amb l'API de Google Gemini

Vegem com es fa una crida bàsica a l'API de Google:

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

# Configurar client amb API key
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

print("📋 Models Gemini disponibles amb la teva API Key:\n")
print("=" * 70)

try:
    # Llistar tots els models
    models = client.models.list()
    
    found_models = False
    
    for model in models:
        # El nou SDK retorna objectes diferents
        model_name = model.name
        
        # Mostrar informació del model
        print(f"\n✅ {model_name}")
        
        if hasattr(model, 'display_name'):
            print(f"   Display Name: {model.display_name}")
        
        if hasattr(model, 'description'):
            print(f"   Description: {model.description}")
        
        # Nom per usar amb CrewAI (eliminar prefix "models/")
        clean_name = model_name.replace("models/", "")
        print(f"   🔧 Usar amb CrewAI: gemini/{clean_name}")
        print("-" * 70)
        
        found_models = True
    
    if not found_models:
        print("⚠️  No s'han trobat models")
        print("   Pot ser que l'API Key no tingui permisos")
    
except Exception as e:
    print(f"❌ Error llistant models: {e}")
    print(f"\n💡 Verifica que la teva API Key és correcta:")
    api_key = os.getenv("GOOGLE_API_KEY")
    if api_key:
        print(f"   API Key: ...{api_key[-4:]}")
    else:
        print("   API Key: NO TROBADA al .env")
    
    print("\n🔍 Detalls de l'error:")
    import traceback
    traceback.print_exc()

In [ ]:
# Una crida bàsica a l'API
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='Explica\'m en una frase què és un díode.'
)

print(response.text)

**Això és programació amb LLMs tradicional.** Però... **això NO és un agent encara!**

Ara veurem per què.

---
## 2. Costos i Rendiment dels Models

Una decisió tècnica important quan treballem amb agents és **quin model utilitzar**. No tots els models són iguals!

### 2.1 Per què importa el cost?

Cada crida a l'API té un cost. Si el teu agent fa:
- 100 crides al dia → Cost diari
- 1,000 crides al dia → Cost considerable
- 10,000 crides al dia → Has de vigilar molt el model!

**Missatge clau:** No sempre necessites el model més potent. Sovint un model més petit i ràpid és millor!

### 2.2 Comparativa de Models Google Gemini (gener 2025)

| Model | Input ($/1M tokens) | Output ($/1M tokens) | Millor per a... |
|-------|---------------------|----------------------|------------------|
| **gemini-2.5-flash** | $0.075 | $0.30 | Tasques ràpides, alt volum, excel·lent qualitat |
| **gemini-2.5-pro** | $1.25 | $5.00 | Tasques complexes, raonament avançat |
| **gemini-2.0-flash-exp** | GRATIS | GRATIS | Experimentació (límits d'ús) |
| **gemini-1.5-flash** | $0.075 | $0.30 | Versió anterior, encara vàlida |
| **gemini-1.5-pro** | $1.25 | $5.00 | Versió anterior del Pro |

**Exemple pràctic:**
- Analitzar un BOM de 500 tokens amb Gemini 2.5 Flash: ~$0.00015 (gairebé gratis!)
- La mateixa tasca amb Gemini 2.5 Pro: ~$0.0025 (17x més car)
- Si fas 10,000 anàlisis/mes: $1.50 vs $25

**📍 Preus actualitzats:** https://ai.google.dev/pricing

### 2.3 Comparativa: Gemini vs OpenAI vs Anthropic

| Proveïdor | Model econòmic | Input | Output | Qualitat |
|-----------|----------------|--------|--------|----------|
| **Google** | Gemini 2.5 Flash | $0.075 | $0.30 | ⭐⭐⭐⭐ |
| **OpenAI** | GPT-4o-mini | $0.15 | $0.60 | ⭐⭐⭐⭐ |
| **Anthropic** | Claude Haiku 4 | $0.25 | $1.25 | ⭐⭐⭐⭐ |

| Proveïdor | Model premium | Input | Output | Qualitat |
|-----------|---------------|--------|--------|----------|
| **Google** | Gemini 2.5 Pro | $1.25 | $5.00 | ⭐⭐⭐⭐⭐ |
| **OpenAI** | GPT-4o | $2.50 | $10.00 | ⭐⭐⭐⭐⭐ |
| **Anthropic** | Claude Sonnet 4.5 | $3.00 | $15.00 | ⭐⭐⭐⭐⭐ |

**Avantatge de Gemini:** Els models més econòmics del mercat amb excel·lent qualitat!

**Recursos per comparar:**
- **Artificial Analysis:** https://artificialanalysis.ai/ (cost vs qualitat vs velocitat)

Si busqueu, hi ha moltes comparatives per internet.

### 2.4 Regla d'or per escollir model

1. **Comença sempre amb Gemini 2.5 Flash**  
   És ràpid, molt barat, i excel·lent qualitat

2. **Puja a Gemini 2.5 Pro només si:**
   - La qualitat de Flash no és suficient
   - La tasca és realment complexa
   - El cost no és un problema

3. **Usa Gemini 2.0 Flash Experimental per:**
   - Proves i desenvolupament (GRATIS!)
   - Prototips inicials
   - Verificar que la solució funciona abans de gastar diners

**Per al nostre projecte d'escandalls:** Gemini 2.5 Flash serà perfecte per a la majoria de tasques!

💡 **Consell pràctic:** Fes sempre A/B testing. Prova la teva tasca amb diferents models i mesura:
- Qualitat del resultat
- Temps de resposta  
- Cost

Després decideix quin és el millor equilibri per al teu cas d'ús.

---
## 3. Frameworks d'Agents: De Baix a Alt Nivell

Abans d'entrar en agents, és important entendre que hi ha **diferents maneres de treballar amb LLMs**, cada una amb el seu nivell d'abstracció.

### 3.1 Les capes de frameworks d'agents

Imagina una piràmide amb diferents capes:

<p align="center">
  <img src="diagrams/ai_stack_architecture.png" width="350">
  <br>
  <em>Stack tecnològic d’aplicacions basades en agents i LLMs</em>
</p>

**A mesura que puges:**
- ✅ Més abstracció (menys codi repetitiu)
- ✅ Més fàcil d'usar
- ✅ Més funcionalitats predefinides
- ⚠️ Menys control sobre els detalls
- ⚠️ Menys flexibilitat

### 3.2 Descripció de cada capa

#### 🔹 Capa 1: APIs REST
**Què és:** Crides HTTP directes a l'API del proveïdor

**Exemple:**
```python
import requests
response = requests.post(
    'https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent',
    headers={'Authorization': f'Bearer {API_KEY}'},
    json={'contents': [{'parts': [{'text': 'Hello'}]}]}
)
```

**Quan usar:**
- ✅ Control total sobre les crides
- ✅ No depens de llibreries de tercers
- ❌ Molt codi repetitiu
- ❌ Gestió manual d'errors, retries, etc.

---

#### 🔹 Capa 2: SDKs de Proveïdors
**Què és:** Llibreries oficials dels proveïdors que simplifiquen les crides

**Exemples:**
- `google-genai` (Google)
- `openai` (OpenAI)
- `anthropic` (Anthropic)

**Codi:**
```python
from google import genai
client = genai.Client(api_key=API_KEY)
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='Hello'
)
```

**Quan usar:**
- ✅ Més simple que crides REST
- ✅ Gestió automàtica d'errors
- ✅ Millor documentació
- ❌ Lligat a un proveïdor específic
- ❌ Encara necessites gestionar molt manualment

---

#### 🔹 Capa 3: Frameworks de Workflows
**Què és:** Frameworks per construir cadenes i grafs de LLMs

**Exemples:**
- **LangChain** - Cadenes de LLMs, templates, memòria
- **LangGraph** - Grafs amb nodes i transicions
- **LlamaIndex** - Enfocada en RAG i indexació

**Característiques:**
- ✅ Multi-proveïdor (OpenAI, Google, Anthropic...)
- ✅ Components reutilitzables (chains, retrievers, etc.)
- ✅ Ecosistema gran de plugins
- ⚠️ Corba d'aprenentatge mitjana
- ⚠️ Molt flexible però pot ser complex

**Quan usar:**
- Necessites workflows complexos amb branching
- Vols mantenir la flexibilitat de canviar de proveïdor
- Treballes amb RAG (Retrieval Augmented Generation)

---

#### 🔹 Capa 4: Frameworks d'Agents
**Què és:** Frameworks especialitzats en crear **equips d'agents** autònoms

**Exemples:**
- **CrewAI** ← **El que utilitzarem!**
- **AutoGen** (Microsoft)
- **Agency Swarm**

**Característiques:**
- ✅ Abstracció d'alt nivell (menys codi)
- ✅ Conceptes d'agents, roles, tasques predefinits
- ✅ Col·laboració entre agents automàtica
- ✅ Multi-proveïdor (funciona amb qualsevol LLM)
- ⚠️ Menys control sobre detalls baixos
- ⚠️ Opinat (et força a fer les coses d'una manera)

**Quan usar:**
- Vols crear equips d'agents ràpidament
- La teva tasca es pot dividir en roles clars
- Prefereixes menys codi i més productivitat

---

#### 🔹 Capa 5: No-Code / Low-Code
**Què és:** Plataformes visuals per crear workflows i agents **sense programar** (o amb programació mínima)

**Categoria A: Plataformes d'automatització generals**
- **n8n** - Workflows visuals amb nodes, integració amb 400+ serveis
- **Zapier** - Automatitzacions simples entre apps ("si això, aleshores allò")
- **Make (Integromat)** - Similar a Zapier però més visual i potent

**Categoria B: Eines visuals específiques per LLMs**
- **Flowise** - UI visual per LangChain (crear flows arrossegant nodes)
- **LangFlow** - Altra UI visual per LangChain amb més features
- **Dify** - Plataforma completa no-code per apps d'IA
- **LangGraph Studio** - UI oficial de LangGraph per crear grafs visuals
- **CrewAI Studio** - UI per CrewAI

**Exemple visual (n8n):**
```
[Trigger: Email rebut] → [Node: Gemini analitza] → [Decision: Urgent?]
                                                          |
                                                    Sí → [Slack notifica]
                                                    No → [Google Sheets guarda]
```

**Característiques:**
- ✅ Zero programació (o mínima)
- ✅ Interface visual (drag & drop)
- ✅ Perfecte per prototips ràpids
- ✅ Accessible per no-programadors
- ⚠️ Limitat per el que ofereix la plataforma
- ⚠️ Difícil de versionar i debugar en producció
- ⚠️ Pot ser car a escala (pricing per execucions)

**Quan usar:**
- Prototips ràpids per validar idees
- Automatitzacions senzilles internes
- Proves de concepte abans de programar
- Equips sense programadors que necessiten automatitzar

**⚠️ Limitacions per producció:**
- Menys control sobre logs i traces
- Dificultat per integrar amb CI/CD
- Vendor lock-in (quedar lligat a la plataforma)
- Escalabilitat limitada

### 3.3 Per què hem escollit CrewAI?

Per a aquesta formació hem escollit **CrewAI** (capa 4) per diverses raons:

**✅ Avantatges per la formació:**
1. **Alt nivell d'abstracció** - Menys codi per aprendre conceptes d'agents
2. **Multi-proveïdor** - Podeu usar Google Gemini, OpenAI, o qualsevol altre
3. **Conceptes clars** - Agents, Roles, Tasks, Crews són fàcils d'entendre
4. **Productiu** - Podeu crear sistemes complexos ràpidament
5. **Comunitat activa** - Molt suport i exemples

**⚠️ Desavantatges (per ser honest):**
1. **Menys control** - Algunes decisions les pren CrewAI per tu
2. **Opinat** - Has de seguir la filosofia de Crew/Agent/Task
3. **Debugging** - Més difícil veure exactament què passa per sota

**🎯 Per al nostre cas d'ús (valoració d'escandalls):**
- Tenim rols clars: Analitzador BOM → Buscador de preus → Comparador → Generador d'informes
- Volem productivitat: Crear el sistema ràpidament
- Volem flexibilitat de proveïdor: Podeu canviar de Gemini a GPT si cal

**CrewAI és perfecte per això!** 🎯

### 3.4 Taula comparativa per decidir

| Si necessites... | Usa... |
|------------------|--------|
| Control total sobre cada byte | APIs REST |
| Crides simples a un LLM | SDK del proveïdor |
| Workflows complexos amb branching | LangGraph |
| RAG i cerca vectorial | LlamaIndex |
| Equips d'agents col·laborant | **CrewAI** ← Nosaltres! |
| Converses multi-agent complexes | AutoGen |

**💡 Consell:** Comença amb l'eina més simple que resolgui el teu problema. Sempre pots baixar de nivell si necessites més control!

## 4. Workflows vs Agents: La Diferència Clau

Aquesta és la secció més important conceptualment. Entendre aquesta diferència canviarà com penses sobre IA.

### 4.1 Què és un Workflow?

Un **workflow** (o patró agèntic) és una **seqüència predefinida** de crides a LLMs.

Tu, com a programador, decideixes:
- Quins LLMs es criden
- En quin ordre
- Amb quines condicions

**Exemples de workflows:**

#### 🔗 1. Prompt Chaining

**Descompondre una tasca en sub-tasques fixes**

<p align="center">
  <img src="diagrams/prompt_chaining.png" width="450">
  <br>
  <em>Exemple de prompt chaining</em>
</p>

**Exemple:** 
1. LLM1 genera una pregunta tècnica sobre un component
2. Gate decideix si la pregunta és vàlida
3. LLM2 respon la pregunta tècnica
4. LLM3 valida que la resposta sigui correcta

**Característiques:**
- ✅ Seqüència lineal i predictible
- ✅ Fàcil de debugar
- ❌ No s'adapta a situacions noves

**Analogia JCM:** Com una cadena de muntatge - cada estació fa sempre el mateix.

#### 🔀 2. Routing

**Dirigir l'input a un LLM especialitzat**
<p align="center">
  <img src="diagrams/llm_router_experts.png" width="550">
  <br>
  <em>Selecció d’LLMs experts per domini mitjançant un router</em>
</p>

**Exemple:**
- Router classifica la consulta (component electrònic, peça mecànica, software)
- Envia a l'LLM expert corresponent

**Característiques:**
- ✅ Separació de concerns
- ✅ LLMs especialitzats per domini
- ❌ Les rutes són fixes (no pot crear rutes noves)

**Analogia JCM:** Com distribuir queries de suport a diferents departaments.

#### ⚡ 3. Parallelization

**Executar múltiples sub-tasques simultàniament**
<p align="center">
  <img src="diagrams/coordinator_flow_diagram.png" width="550">
  <br>
  <em>Selecció d’LLMs experts per domini mitjançant un router</em>
</p>

**Exemple BOM:**
- Analitzar un component des de 3 perspectives diferents en paral·lel
- Agregar els resultats en un informe complet

**Característiques:**
- ✅ Més ràpid (paral·lel)
- ✅ Perspectives múltiples
- ❌ Complexitat en l'agregació

**Analogia JCM:** Com fer que 3 enginyers treballin en el mateix disseny simultàniament.

#### 🎭 4. Orchestrator-Worker

**Tasques complexes descompostes dinàmicament**

<p align="center">
  <img src="diagrams/orchestrator_diagram.png" width="500">
  <br>
  <em>Selecció d’LLMs experts per domini mitjançant un router</em>
</p>

**Exemple:**
- Orchestrator decideix quines parts d'un BOM analitzar
- Workers executen les anàlisis en paral·lel
- Synthesizer combina els resultats

**Característiques:**
- ✅ Descomposició dinàmica de tasques
- ✅ Escalable
- ❌ Encara és un workflow (flux predeterminat)

#### 🔄 5. Evaluator-Optimizer

**Validar i millorar iterativament**

<p align="center">
  <img src="diagrams/evaluator_diagram.png" width="350">
  <br>
  <em>Bucle de generació i avaluació amb feedback en cas de rebuig</em>
</p>

**Exemple:**
- Generator crea una llista de components alternatius
- Evaluator valida que compleixin especificacions
- Si no, torna a generar amb feedback

**Característiques:**
- ✅ Auto-millora
- ✅ Control de qualitat
- ❌ Pot ser lent (múltiples iteracions)

**Analogia JCM:** Com el procés de revisió de disseny fins que passa QA.

### 4.2 Aleshores... què és un Agent?

Un **agent** és fonamentalment diferent. 

_"An AI agent is a system that takes a goal, makes decisions, and acts to move work forward."_

És a dir, l'agent:
- **Percep l'entorn** (llegeix informació disponible)
- **Pren decisions autònomes** sobre què fer
- **Actua** (executa accions, crida tools)
- **Rep feedback** de l'entorn
- **Itera** fins assolir l'objectiu o decidir parar

**Esquema d'un agent:**

<p align="center">
  <img src="diagrams/human_llm_agent_environment.png" width="350">
  <br>
  <em>Bucle d’interacció entre una persona, un agent LLM i l’entorn</em>
</p>

**L'agent decideix:**
- Quina acció fer (de les disponibles)
- Quan parar o continuar
- Com interpretar el feedback
- Com adaptar l'estratègia

**Característiques clau:**
- ✅ Pren decisions autònomes
- ✅ S'adapta segons el feedback
- ✅ Pot usar eines (tools) per interactuar amb l'entorn
- ✅ Aprèn de l'experiència (dins d'una execució)
- ⚠️ Més difícil de controlar
- ⚠️ Pot ser impredictible

### 4.3 Comparació directa

| Aspecte | Workflow | Agent |
|---------|----------|-------|
| **Control** | Tu decideixes el flux | L'agent decideix |
| **Adaptabilitat** | Flux fix predefinit | S'adapta dinàmicament |
| **Predictibilitat** | Molt predictible | Menys predictible |
| **Complexitat** | Més simple | Més complex |
| **Autonomia** | Zero autonomia | Alta autonomia |
| **Cas d'ús** | Processos coneguts | Situacions obertes |
| **Debugging** | Fàcil (flux fix) | Difícil (decisions dinàmiques) |
| **Cost** | Predictible | Variable segons decisions |

**Regla d'or:**
- Si **saps exactament** els passos → Workflow
- Si l'agent ha de **decidir** què fer → Agent

### 💡 Reflexió

Pensa en el cas d'ús de valoració d'escandalls que farem més endavant:

**Escenari:**
1. Reps un BOM amb 50 components
2. Has de buscar preus a diversos proveïdors
3. Algunes peces poden no estar disponibles
4. Potser cal buscar alternatives
5. Generar un informe final

**Pregunta:** Creus que hauria de ser un workflow o un agent? Per què?

**Pista:** Pensa en:
- Són els passos sempre els mateixos?
- Pot haver-hi situacions inesperades?
- Cal prendre decisions segons el que es trobi?

*Pensaràs en la resposta quan arribem al cas pràctic...*

---
## 5. Mans a l'Obra: El Teu Primer Crew amb CrewAI

Ara que entens la teoria, anem a crear el nostre primer **Crew** amb CrewAI i Google Gemini.

### 5.1 Conceptes bàsics de CrewAI

CrewAI organitza els agents en una metàfora d'**equip de treball**:

**🎭 Agent:**
- Un membre de l'equip amb un **rol** específic
- Té un **objectiu** (goal)
- Té un **backstory** que defineix la seva expertesa
- Pot tenir **tools** per executar accions

**📋 Task:**
- Una tasca concreta a realitzar
- Assignada a un agent específic
- Té una **descripció** clara
- Té un **resultat esperat** (expected_output)

**👥 Crew:**
- L'equip complet
- Agrupa agents i tasques
- Defineix com col·laboren (process)
- Executa el workflow

**Analogia JCM:**
- **Crew** = Departament de R+D
- **Agents** = Enginyers amb especialitats diferents
- **Tasks** = Projectes assignats a cada enginyer

### 5.2 El nostre primer crew: Analitzador Tècnic

Crearem un crew super simple: **1 agent, 1 task**.

**Objectiu:** Respondre preguntes tècniques sobre components electrònics.

In [ ]:
# Imports necessaris
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
import os

# Carregar API key
load_dotenv(override=True)

print("✅ CrewAI importat correctament!")

#### Pas 1: Crear l'Agent

In [ ]:
# Definir l'agent
agent_tecnic = Agent(
    role="Enginyer Electrònic Senior",
    goal="Proporcionar informació tècnica precisa sobre components electrònics",
    backstory="""
    Ets un enginyer electrònic amb 15 anys d'experiència a JCM Technologies.
    Has treballat en disseny de circuits de control per a portes automàtiques
    i tens un coneixement profund de components com relés, sensors i actuadors.
    
    La teva especialitat és explicar conceptes tècnics de manera clara i precisa,
    sempre amb exemples pràctics del món real.
    """,
    llm="gemini/gemini-2.5-flash",  # Model de Google Gemini
    verbose=True  # Per veure què fa l'agent
)

print("✅ Agent 'Enginyer Electrònic Senior' creat!")

**💡 Nota sobre el model:**

CrewAI usa el format `proveïdor/model` per especificar models:
- `gemini/gemini-2.5-flash` → Google Gemini
- `openai/gpt-4o-mini` → OpenAI
- `anthropic/claude-sonnet-4.5` → Anthropic

Això fa CrewAI **multi-proveïdor**! Pots canviar de model canviant una línia.

#### Pas 2: Crear la Task

In [ ]:
# Definir la tasca
task_explicar = Task(
    description="""
    Explica què és un relé i per què s'utilitza en sistemes de control
    de portes automàtiques.
    
    Inclou:
    - Definició tècnica del component
    - Com funciona internament
    - Avantatges d'utilitzar-lo en aquest context
    - Un exemple pràctic d'aplicació
    """,
    expected_output="""
    Una explicació tècnica clara de 3-4 paràgrafs que cobreixi tots els punts,
    escrita de manera professional però accessible.
    """,
    agent=agent_tecnic  # Assignem la tasca a l'agent
)

print("✅ Task 'Explicar relé' creada i assignada!")

#### Pas 3: Crear i executar el Crew

In [ ]:
# Crear el crew
crew = Crew(
    agents=[agent_tecnic],
    tasks=[task_explicar],
    verbose=True  # Per veure el procés complet
)

print("✅ Crew creat amb 1 agent i 1 task!")

In [ ]:
# Executar el crew!
print("🚀 Executant el crew...\n")
print("=" * 70)

result = crew.kickoff()

print("=" * 70)
print("\n✅ Crew executat amb èxit!\n")
print("📄 RESULTAT:\n")
print(result)

### 5.3 Què ha passat?

Quan executem `crew.kickoff()`:

1. **CrewAI inicialitza** l'agent amb el seu rol i backstory
2. **Envia la task** a l'agent
3. **L'agent usa Gemini** per generar la resposta
4. **Valida** que la resposta compleixi l'expected_output
5. **Retorna** el resultat final

**💡 Diferència clau amb un LLM tradicional:**

```python
# LLM tradicional (1 crida)
response = client.models.generate_content("Explica què és un relé")

# CrewAI Agent (pot fer múltiples crides si cal)
result = crew.kickoff()  # L'agent decideix quantes crides fer
```

L'agent pot fer **múltiples iteracions** fins que estigui satisfet amb el resultat!

### 5.4 Provant amb diferents preguntes

Ara modifiquem la task per fer una altra pregunta:

In [ ]:
# Nova task amb una pregunta diferent
task_sensors = Task(
    description="""
    Compara les fotocèl·ules i les bandes de seguretat
    per a detecció d'obstacles en portes automàtiques.
    
    Inclou:
    - Com funciona cada tipus de sensor
    - Avantatges i desavantatges de cada un
    - En quines situacions és millor usar cada tipus
    """,
    expected_output="""
    Una comparativa tècnica de 4-5 paràgrafs que ajudi a decidir
    quin sensor utilitzar segons el cas d'ús.
    """,
    agent=agent_tecnic
)

# Crear nou crew amb la nova task
crew_sensors = Crew(
    agents=[agent_tecnic],
    tasks=[task_sensors],
    verbose=True
)

# Executar
result_sensors = crew_sensors.kickoff()
print("\n📄 RESULTAT:\n")
print(result_sensors)

### 💡 Reflexió Important

**Pregunta:** L'exemple que acabem de crear, és realment un "agent" autònom?

Pensa en:
- L'agent pren decisions?
- Interactua amb l'entorn?
- Utilitza tools?
- S'adapta segons feedback?

**Resposta:** Encara **NO** és un agent complet! És més aviat un **LLM amb context millorat**.

Per ser un agent real necessita:
- ✅ **Tools** per interactuar amb l'entorn (llegir fitxers, cridar APIs, etc.)
- ✅ **Capacitat de decisió** sobre quina tool usar
- ✅ **Iteració** basada en els resultats de les tools

**Això ho veurem a la Part 2!** Ara anem a aplicar-ho al cas del BOM.

## 6. Cas Pràctic: Crew Analitzador de BOM

Ara aplicarem tot el que hem après a un cas real: analitzar un Bill of Materials (BOM) utilitzant CrewAI.

### 6.1 Què és un BOM?

Un **Bill of Materials** (Escandall) és una llista estructurada de tots els components necessaris per fabricar un producte.

**Exemple simplificat:**

| Part Number | Description | Quantity | Unit |
|-------------|-------------|----------|------|
| RES-001 | Resistència 10kΩ | 5 | pcs |
| CAP-002 | Condensador 100nF | 3 | pcs |
| LED-003 | LED vermell 5mm | 2 | pcs |
| IC-004 | Microcontrolador ATmega328 | 1 | pcs |

**A JCM Technologies:**
- Cada producte (control de porta, sensor, etc.) té el seu BOM
- Cal analitzar-los per: costejat, planificació, compres, qualitat

### 6.2 Objectiu: Crew que analitza un BOM

Volem un crew que pugui:
1. Llegir un BOM (text o dades estructurades)
2. Identificar els components
3. Agrupar-los per categoria
4. Detectar possibles problemes o inconsistències
5. **Retornar dades estructurades** (no text lliure!)

El punt 5 és clau: volem poder processar el resultat programàticament!

### 6.3 BOM simplificat per la demo

In [ ]:
# BOM simplificat en format text
bom_simple = """
BILL OF MATERIALS - Control Board v2.1
=======================================

1. RES-10K - Resistència 10kΩ 1/4W - Qty: 5
2. RES-1K - Resistència 1kΩ 1/4W - Qty: 3  
3. CAP-100N - Condensador ceràmic 100nF - Qty: 4
4. CAP-10U - Condensador electrolític 10µF - Qty: 2
5. LED-RED - LED vermell 5mm - Qty: 2
6. LED-GREEN - LED verd 5mm - Qty: 1
7. RELAY-5V - Relé 5V 10A SPDT - Qty: 3
8. IC-ATMEGA - Microcontrolador ATmega328P - Qty: 1
9. CONN-RJ45 - Connector RJ45 - Qty: 1
10. PCB - Placa de circuit imprès - Qty: 1
"""

print(bom_simple)

### 6.4 Definir l'esquema de sortida amb Pydantic

Abans de crear l'agent, definim **exactament** quin format volem a la sortida.

**Per què Pydantic?**
- ✅ Validació automàtica de tipus
- ✅ Errors clars si les dades són incorrectes
- ✅ Conversió fàcil a dict/JSON
- ✅ Integració directa amb CrewAI

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Dict

class Component(BaseModel):
    """Un component del BOM"""
    part_number: str = Field(description="Codi identificador del component")
    description: str = Field(description="Descripció tècnica del component")
    quantity: int = Field(description="Quantitat necessària", gt=0)
    category: str = Field(description="Categoria del component (Resistència, Condensador, etc.)")

class BOMAnalysis(BaseModel):
    """Anàlisi estructurada d'un BOM"""
    product_name: str = Field(description="Nom del producte")
    components: List[Component] = Field(description="Llista de tots els components")
    total_components: int = Field(description="Total de línies de components")
    categories_summary: Dict[str, int] = Field(description="Resum de components per categoria")
    observations: List[str] = Field(description="Possibles problemes o notes rellevants")

print("✅ Esquemes Pydantic definits!")
print("\nEsquema Component:")
print(Component.model_json_schema())
print("\nEsquema BOMAnalysis:")
print(BOMAnalysis.model_json_schema())

### 6.5 Crear l'Agent Analitzador de BOM

In [ ]:
from crewai import Agent, Task, Crew

# Agent especialitzat en analitzar BOMs
agent_bom_analyzer = Agent(
    role="Analitzador de Bills of Materials",
    goal="Analitzar BOMs i extreure informació estructurada sobre components electrònics",
    backstory="""
    Ets un especialista en gestió de BOMs amb 10 anys d'experiència a JCM Technologies.
    
    La teva expertesa inclou:
    - Identificació precisa de components electrònics
    - Classificació per categories (resistències, condensadors, semiconductors, etc.)
    - Detecció d'inconsistències o components estranys
    - Anàlisi de consistència de BOMs
    
    Ets meticulós i sempre proporciones dades estructurades i verificades.
    """,
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

print("✅ Agent Analitzador de BOM creat!")

### 6.6 Crear la Task amb sortida estructurada

Aquí ve la part important: especifiquem que volem la sortida en format **Pydantic**!

In [ ]:
# Task amb sortida estructurada
task_analyze_bom = Task(
    description=f"""
    Analitza el següent BOM i extreu tota la informació de manera estructurada:
    
    {bom_simple}
    
    Per a cada component del BOM, has de:
    1. Extreure el part number exacte
    2. Extreure la descripció completa
    3. Extreure la quantitat
    4. Classificar-lo en la categoria apropiada:
       - Resistència
       - Condensador
       - Semiconductor (LED, IC, etc.)
       - Connector
       - Electromecànic (relés)
       - PCB
       - Altres
    
    També has de:
    - Comptar el total de components diferents (línies del BOM)
    - Crear un resum de quants components hi ha per cada categoria
    - Identificar possibles observacions (components poc comuns, quantitats estranyes, etc.)
    """,
    expected_output="""
    Un objecte BOMAnalysis amb:
    - Nom del producte identificat
    - Llista completa de components amb tota la informació
    - Total de components
    - Resum per categories
    - Observacions rellevants
    """,
    agent=agent_bom_analyzer,
    output_pydantic=BOMAnalysis  # ← Aquí especifiquem el format Pydantic!
)

print("✅ Task amb sortida estructurada (Pydantic) creada!")

**💡 La clau està a `output_pydantic=BOMAnalysis`**

Això li diu a CrewAI:
1. Genera l'esquema JSON del model Pydantic
2. Envia'l a Gemini perquè sàpiga exactament quin format retornar
3. Valida automàticament que la resposta compleixi l'esquema
4. Retorna un objecte Python (no text!)

### 6.7 Crear i executar el Crew

In [ ]:
# Crear el crew
crew_bom = Crew(
    agents=[agent_bom_analyzer],
    tasks=[task_analyze_bom],
    verbose=True
)

print("✅ Crew BOM Analyzer creat!")

### 💡 Nota sobre `verbose=True`

Has vist que posem `verbose=True` al crear el crew. Això mostra **traces** 
de l'execució: què pensa l'agent, quines decisions pren, quines crides fa al LLM.

Per ara, només observa la sortida. A la **Part 2**, quan treballem amb **tools**, 
les traces seran fonamentals per entendre:
- Quina tool ha decidit usar l'agent
- Per què ha pres aquesta decisió  
- Què ha obtingut de cada tool
- Com itera basant-se en els resultats

**Consell:** Mantén sempre `verbose=True` durant el desenvolupament. 
És la teva millor eina de debugging!

In [ ]:
# Executar el crew!
print("🚀 Analitzant el BOM...\n")
print("=" * 70)

result = crew_bom.kickoff()

print("=" * 70)
print("\n✅ Anàlisi completada!\n")

### 6.8 Processar el resultat estructurat

Ara tenim un **objecte Python validat**, no text! Podem processar-lo directament:

In [ ]:
# El resultat és un objecte BOMAnalysis!
bom_analysis = result.pydantic

print(f"📦 Producte: {bom_analysis.product_name}")
print(f"📊 Total de components: {bom_analysis.total_components}")
print("\n📂 Components per categoria:")
for categoria, count in bom_analysis.categories_summary.items():
    print(f"  - {categoria}: {count}")

if bom_analysis.observations:
    print("\n📝 Observacions:")
    for obs in bom_analysis.observations:
        print(f"  • {obs}")

### 6.9 Visualitzar els components amb pandas

In [ ]:
import pandas as pd

# Convertir components a DataFrame
df_components = pd.DataFrame([c.model_dump() for c in bom_analysis.components])

print("\n📋 Taula de components:\n")
display(df_components)

# Resum per categories
print("\n📊 Resum per categories:\n")
df_categories = pd.DataFrame([
    {"Categoria": cat, "Quantitat de línies": count} 
    for cat, count in bom_analysis.categories_summary.items()
])
display(df_categories)

### 6.10 Exportar a diferents formats

Com que tenim dades estructurades, podem exportar-les fàcilment:

In [ ]:
# Exportar a JSON
json_output = bom_analysis.model_dump_json(indent=2)
print("JSON format:\n")
print(json_output[:500] + "...\n")  # Mostrem només els primers 500 caràcters

# Guardar a fitxer
with open('data/bom_analysis.json', 'w', encoding='utf-8') as f:
    f.write(json_output)
print("✅ Guardat a: bom_analysis.json")

# Exportar components a Excel
df_components.to_excel('data/bom_components.xlsx', index=False)
print("✅ Components guardats a: bom_components.xlsx")

### 💡 Diferència amb l'enfocament tradicional

**Sense structured outputs (text lliure):**
```python
result = "El BOM conté 10 components: 2 resistències, 2 condensadors..."
# Ara hauries de fer parsing del text amb regex 😱
```

**Amb structured outputs (Pydantic):**
```python
result = BOMAnalysis(components=[...], total_components=10, ...)
# Accés directe a dades validades! 🎉
print(result.total_components)  # 10
print(result.components[0].category)  # "Resistència"
```

**Avantatges:**
- ✅ No cal parsing de text
- ✅ Validació automàtica de tipus
- ✅ Errors clars si alguna cosa va malament
- ✅ Autocompletat al IDE
- ✅ Fàcil d'integrar amb bases de dades, APIs, etc.

### 6.11 Reflexió final sobre Agents vs Workflows

**Pregunta:** El que acabem de crear, és un workflow o un agent?

**Anàlisi:**
- ✅ Té un **rol** específic (Analitzador de BOM)
- ✅ Retorna **dades estructurades** 
- ❌ No pren **decisions** autònomes
- ❌ No usa **tools** per interactuar amb l'entorn
- ❌ No **itera** basant-se en feedback

**Resposta:** És més proper a un **workflow sofisticat** que a un veritable agent autònom.

**Per convertir-lo en un agent real necessitaríem:**
1. **Tools** per:
   - Llegir fitxers Excel reals
   - Buscar informació de components a internet
   - Consultar bases de dades de proveïdors
   
2. **Capacitat de decisió:**
   - "Aquest component sembla estrany, busco més informació?"
   - "Necessito validar aquest preu amb un proveïdor?"
   
3. **Iteració basada en resultats:**
   - Si no troba info d'un component → busca alternatives
   - Si detecta inconsistència → fa comprovacions addicionals

**Això ho construirem a la Part 2: Tools i Function Calling!** 🚀

---
## Reflexió Final de la Part 1

### 🎓 Què hem après?

1. ✅ **LLMs i APIs**
   - Com cridar Google Gemini des de Python
   - Diferència entre usar l'API directament vs frameworks

2. ✅ **Costos i models**
   - Gemini és molt econòmic (Flash: $0.075/1M tokens)
   - Comparativa amb OpenAI i Anthropic
   - Regla d'or: començar amb models econòmics

3. ✅ **Capes de frameworks**
   - Des d'APIs REST fins a No-code
   - Per què hem escollit CrewAI (capa 4)
   - Quan usar cada capa

4. ✅ **Workflows vs Agents**
   - 5 patrons agèntics comuns
   - Diferència clau: control vs autonomia
   - Quan usar cada un

5. ✅ **CrewAI bàsic**
   - Anatomia: Agent, Task, Crew
   - Primer crew simple (1 agent, 1 task)
   - Com configurar models de Gemini

6. ✅ **Structured Outputs amb Pydantic**
   - Definir esquemes amb BaseModel
   - Integració amb CrewAI (`output_pydantic`)
   - Processar dades estructurades
   - Cas pràctic: Analitzador de BOM

### 🚀 Pròxims passos

A la **Part 2: Tools i Function Calling** veurem:

- 🔧 **Crear tools personalitzades**
  - Tool per llegir fitxers Excel (BOMs reals)
  - Tool per "enviar" emails (via Gmail API)
  - Tool per consultar APIs externes

- 🤖 **Agents que usen tools**
  - Com l'agent decideix quina tool usar
  - Iteració basada en resultats de tools
  - Agents veritablement autònoms

- 👥 **Múltiples agents col·laborant**
  - Un agent per analitzar, un altre per buscar preus
  - Com es passen informació entre ells
  - Processos Sequential vs Hierarchical

A la **Part 3: Cas Pràctic Complet** integrarem tot:
- Sistema multi-agent per valoració d'escandalls
- Workflow complet: BOM → Preus → Comparació → Informe
- El vostre projecte real! 🎯

---
<table style="margin: 20px 0; text-align: left; width:100%; border: 2px solid #27ae60;">
    <tr>
        <td style="width: 150px; vertical-align: middle; padding: 20px;">
            <img src="https://raw.githubusercontent.com/googlefonts/noto-emoji/main/png/128/emoji_u1f389.png" width="120" style="display: block;" />
        </td>
        <td style="padding: 20px;">
            <h2 style="color:#27ae60; margin: 0;">Enhorabona!</h2>
            <p style="color:#333; margin: 10px 0;">Has completat la Part 1 dels fonaments d'agents d'IA amb CrewAI i Google Gemini.</p>
            <p style="color:#ff7800; margin: 10px 0 0 0;"><strong>Proper pas:</strong> Part 2 - Tools i Function Calling</p>
        </td>
    </tr>
</table>